In [28]:
!pip install dlib

In [29]:
!pip install imutils

In [30]:
!pip install pyttsx3

In [31]:
!pip install keras

In [32]:
!pip install tensorflow

In [11]:
import dlib
import urllib.request

# 다운로드 링크
url = "http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2"

# 파일 다운로드
urllib.request.urlretrieve(url, "shape_predictor_68_face_landmarks.dat.bz2")

# 압축 해제
import bz2

with open("shape_predictor_68_face_landmarks.dat", "wb") as f_out, bz2.open("shape_predictor_68_face_landmarks.dat.bz2", "rb") as f_in:
    f_out.write(f_in.read())

# 파일 닫기
f_out.close()


## 실행

In [33]:
import cv2
import dlib
import numpy as np
from imutils import face_utils
from keras.models import load_model

IMG_SIZE = (34, 26)

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

model = load_model('models/2023_12_27_15_42_17.h5')
model.summary()

def crop_eye(img, eye_points):
    x1, y1 = np.amin(eye_points, axis=0)
    x2, y2 = np.amax(eye_points, axis=0)
    cx, cy = (x1 + x2) / 2, (y1 + y2) / 2

    w = (x2 - x1) * 1.2
    h = w * IMG_SIZE[1] / IMG_SIZE[0]

    margin_x, margin_y = w / 2, h / 2

    min_x, min_y = int(cx - margin_x), int(cy - margin_y)
    max_x, max_y = int(cx + margin_x), int(cy + margin_y)

    eye_rect = np.rint([min_x, min_y, max_x, max_y]).astype(int)

    eye_img = gray[eye_rect[1]:eye_rect[3], eye_rect[0]:eye_rect[2]]

    return eye_img, eye_rect

capture = cv2.VideoCapture(0)  # 디바이스 아이디
capture.set(cv2.CAP_PROP_FRAME_WIDTH, 2560)
capture.set(cv2.CAP_PROP_FRAME_HEIGHT, 1440)

while True:
    ret, img_ori = capture.read()
    img = img_ori.copy()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    faces = detector(gray)

    if len(faces) > 0:
        face = faces[0]
        shapes = predictor(gray, face)
        shapes = face_utils.shape_to_np(shapes)

        eye_img_l, eye_rect_l = crop_eye(gray, eye_points=shapes[36:42])
        eye_img_r, eye_rect_r = crop_eye(gray, eye_points=shapes[42:48])

        eye_img_l = cv2.resize(eye_img_l, dsize=IMG_SIZE)
        eye_img_r = cv2.resize(eye_img_r, dsize=IMG_SIZE)
        eye_img_r = cv2.flip(eye_img_r, flipCode=1)

        #cv2.imshow('l', eye_img_l)
        #cv2.imshow('r', eye_img_r)

        eye_input_l = eye_img_l.copy().reshape((1, IMG_SIZE[1], IMG_SIZE[0], 1)).astype(np.float32) / 255.
        eye_input_r = eye_img_r.copy().reshape((1, IMG_SIZE[1], IMG_SIZE[0], 1)).astype(np.float32) / 255.

        pred_l = model.predict(eye_input_l)
        pred_r = model.predict(eye_input_r)

        # visualize
        state_l = 'O %.1f' if pred_l > 0.1 else '- %.1f'
        state_r = 'O %.1f' if pred_r > 0.1 else '- %.1f'

        state_l = state_l % pred_l
        state_r = state_r % pred_r

        cv2.rectangle(img, pt1=tuple(eye_rect_l[0:2]), pt2=tuple(eye_rect_l[2:4]), color=(255, 255, 255), thickness=2)
        cv2.rectangle(img, pt1=tuple(eye_rect_r[0:2]), pt2=tuple(eye_rect_r[2:4]), color=(255, 255, 255), thickness=2)

        cv2.putText(img, state_l, tuple(eye_rect_l[0:2]), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        cv2.putText(img, state_r, tuple(eye_rect_r[0:2]), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

    cv2.imshow('result', img)

    key = cv2.waitKey(1)
    if key == 27:  # ESC 키를 누르면 종료
        break

capture.release()
cv2.destroyAllWindows()
cv2.waitKey(1)



Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 26, 34, 1)]       0         
                                                                 
 conv2d_3 (Conv2D)           (None, 26, 34, 32)        320       
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 13, 17, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_4 (Conv2D)           (None, 13, 17, 64)        18496     
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 6, 8, 64)          0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 6, 8, 128)         73

-1

## 음성알림 추가

In [1]:
import cv2
import dlib
import numpy as np
from imutils import face_utils
from tensorflow.keras.models import load_model
import pyttsx3
import time

IMG_SIZE = (34, 26)

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

model = load_model('models/2024_01_05_14_16_21.h5')
model.summary()

# Text-to-speech engine 초기화
engine = pyttsx3.init()
engine.setProperty('rate', 150)  # 음성 속도 조절

def speak(text):
    engine.say(text)
    engine.runAndWait()

def crop_eye(img, eye_points):
    x1, y1 = np.amin(eye_points, axis=0)
    x2, y2 = np.amax(eye_points, axis=0)
    cx, cy = (x1 + x2) / 2, (y1 + y2) / 2

    w = (x2 - x1) * 1.2
    h = w * IMG_SIZE[1] / IMG_SIZE[0]

    margin_x, margin_y = w / 2, h / 2

    min_x, min_y = int(cx - margin_x), int(cy - margin_y)
    max_x, max_y = int(cx + margin_x), int(cy + margin_y)

    eye_rect = np.rint([min_x, min_y, max_x, max_y]).astype(int)

    eye_img = gray[eye_rect[1]:eye_rect[3], eye_rect[0]:eye_rect[2]]

    return eye_img, eye_rect

capture = cv2.VideoCapture(0)  # 디바이스 아이디
capture.set(cv2.CAP_PROP_FRAME_WIDTH, 2560)
capture.set(cv2.CAP_PROP_FRAME_HEIGHT, 1440)

prev_time = time.time()  # 이전에 눈을 감은 시간

# 눈 깜빡임 주기 확인을 위한 변수들
blink_threshold = 3  # 눈을 깜빡이지 않아도 작동할 시간
blink_counter = 0    # 눈을 깜빡인 시간을 측정하는 카운터

while True:
    ret, img_ori = capture.read()
    img = img_ori.copy()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    faces = detector(gray)

    if len(faces) > 0:
        face = faces[0]
        shapes = predictor(gray, face)
        shapes = face_utils.shape_to_np(shapes)

        eye_img_l, eye_rect_l = crop_eye(gray, eye_points=shapes[36:42])
        eye_img_r, eye_rect_r = crop_eye(gray, eye_points=shapes[42:48])

        eye_img_l = cv2.resize(eye_img_l, dsize=IMG_SIZE)
        eye_img_r = cv2.resize(eye_img_r, dsize=IMG_SIZE)
        eye_img_r = cv2.flip(eye_img_r, flipCode=1)

        eye_input_l = eye_img_l.copy().reshape((1, IMG_SIZE[1], IMG_SIZE[0], 1)).astype(np.float32) / 255.
        eye_input_r = eye_img_r.copy().reshape((1, IMG_SIZE[1], IMG_SIZE[0], 1)).astype(np.float32) / 255.

        pred_l = model.predict(eye_input_l)
        pred_r = model.predict(eye_input_r)

        # 눈 깜빡임 주기 확인
        if pred_l < 0.2 and pred_r < 0.2:
            blink_counter += 1
            if blink_counter >= blink_threshold:
                speak("졸음 운전 감지")
                blink_counter = 0  # 초기화
        else:
            blink_counter = 0  # 초기화

        # visualize
        state_l = 'O %.1f' if pred_l > 0.1 else '- %.1f'
        state_r = 'O %.1f' if pred_r > 0.1 else '- %.1f'
        
        state_l = state_l % pred_l
        state_r = state_r % pred_r

        cv2.rectangle(img, pt1=tuple(eye_rect_l[0:2]), pt2=tuple(eye_rect_l[2:4]), color=(255, 255, 255), thickness=2)
        cv2.rectangle(img, pt1=tuple(eye_rect_r[0:2]), pt2=tuple(eye_rect_r[2:4]), color=(255, 255, 255), thickness=2)

        cv2.putText(img, state_l, tuple(eye_rect_l[0:2]), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        cv2.putText(img, state_r, tuple(eye_rect_r[0:2]), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

    cv2.imshow('result', img)

    key = cv2.waitKey(1)
    if key == 27:  # ESC 키를 누르면 종료
        break

capture.release()
cv2.destroyAllWindows()
cv2.waitKey(1)





Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 26, 34, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 26, 34, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 13, 17, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 13, 17, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 6, 8, 64)          0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 6, 8, 128)         738

-1

In [14]:
import cv2
import dlib
import numpy as np
from imutils import face_utils
from keras.models import load_model
import pyttsx3
import time

IMG_SIZE = (34, 26)

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

model = load_model('models/2024_01_05_14_16_21.h5')
model.summary()

# Text-to-speech engine 초기화
engine = pyttsx3.init()
engine.setProperty('rate', 150)  # 음성 속도 조절

def speak(text):
    engine.say(text)
    engine.runAndWait()

def crop_eye(img, eye_points):
    x1, y1 = np.amin(eye_points, axis=0)
    x2, y2 = np.amax(eye_points, axis=0)
    cx, cy = (x1 + x2) / 2, (y1 + y2) / 2

    w = (x2 - x1) * 1.2
    h = w * IMG_SIZE[1] / IMG_SIZE[0]

    margin_x, margin_y = w / 2, h / 2

    min_x, min_y = int(cx - margin_x), int(cy - margin_y)
    max_x, max_y = int(cx + margin_x), int(cy + margin_y)

    eye_rect = np.rint([min_x, min_y, max_x, max_y]).astype(int)

    eye_img = gray[eye_rect[1]:eye_rect[3], eye_rect[0]:eye_rect[2]]

    return eye_img, eye_rect

capture = cv2.VideoCapture(0)  # 디바이스 아이디
capture.set(cv2.CAP_PROP_FRAME_WIDTH, 2560)
capture.set(cv2.CAP_PROP_FRAME_HEIGHT, 1440)

prev_time = time.time()  # 이전에 눈을 감은 시간
blink_threshold = 3  # 눈을 깜빡이지 않아도 작동할 시간
blink_counter = 0    # 눈을 깜빡인 시간을 측정하는 카운터

while True:
    ret, img_ori = capture.read()
    img = img_ori.copy()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # 현재 시간 계산
    current_time = time.time()

    faces = detector(gray)

    if len(faces) > 0:
        face = faces[0]
        shapes = predictor(gray, face)
        shapes = face_utils.shape_to_np(shapes)

        eye_img_l, eye_rect_l = crop_eye(gray, eye_points=shapes[36:42])
        eye_img_r, eye_rect_r = crop_eye(gray, eye_points=shapes[42:48])

        eye_img_l = cv2.resize(eye_img_l, dsize=IMG_SIZE)
        eye_img_r = cv2.resize(eye_img_r, dsize=IMG_SIZE)
        eye_img_r = cv2.flip(eye_img_r, flipCode=1)

        eye_input_l = eye_img_l.copy().reshape((1, IMG_SIZE[1], IMG_SIZE[0], 1)).astype(np.float32) / 255.
        eye_input_r = eye_img_r.copy().reshape((1, IMG_SIZE[1], IMG_SIZE[0], 1)).astype(np.float32) / 255.

        pred_l = model.predict(eye_input_l)
        pred_r = model.predict(eye_input_r)

        # 눈 깜빡임 주기 확인
        if pred_l < 0.2 and pred_r < 0.2:
            blink_counter += 1
            if blink_counter >= blink_threshold:
                speak("졸음 운전 감지")
                blink_counter = 0  # 초기화
        else:
            blink_counter = 0  # 초기화

        # visualize
        state_l = 'O %.1f' if pred_l > 0.1 else '- %.1f'
        state_r = 'O %.1f' if pred_r > 0.1 else '- %.1f'
        
        state_l = state_l % pred_l
        state_r = state_r % pred_r

        cv2.rectangle(img, pt1=tuple(eye_rect_l[0:2]), pt2=tuple(eye_rect_l[2:4]), color=(255, 255, 255), thickness=2)
        cv2.rectangle(img, pt1=tuple(eye_rect_r[0:2]), pt2=tuple(eye_rect_r[2:4]), color=(255, 255, 255), thickness=2)

        cv2.putText(img, state_l, tuple(eye_rect_l[0:2]), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        cv2.putText(img, state_r, tuple(eye_rect_r[0:2]), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

        # 눈이 감겨 있지 않을 때 현재 시간 갱신
        prev_time = current_time
    else:
        # 눈을 5초 이상 감지하지 못할 때 메시지 출력
        elapsed_time = current_time - prev_time
        if elapsed_time > 5 and blink_counter == 0:
            speak("전방 주시 요망")
            blink_counter += 1  # 초기화

    cv2.imshow('result', img)

    key = cv2.waitKey(1)
    if key == 27:  # ESC 키를 누르면 종료
        break

capture.release()
cv2.destroyAllWindows()
cv2.waitKey(1)


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 26, 34, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 26, 34, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 13, 17, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 13, 17, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 6, 8, 64)          0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 6, 8, 128)         73856 

C:\Users\alsgu\AppData\Local\Temp\ipykernel_12740\1587456468.py:93: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  state_l = state_l % pred_l
C:\Users\alsgu\AppData\Local\Temp\ipykernel_12740\1587456468.py:94: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  state_r = state_r % pred_r


1/1 [==============================] - 0s 26ms/step


-1

## 눈 사진 캡쳐

In [8]:
import cv2
import dlib
import numpy as np
from imutils import face_utils
from keras.models import load_model

IMG_SIZE = (34, 26)

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

model = load_model('models/2023_12_26_11_40_10.h5')
model.summary()

def crop_eye(img, eye_points):
    x1, y1 = np.amin(eye_points, axis=0)
    x2, y2 = np.amax(eye_points, axis=0)
    cx, cy = (x1 + x2) / 2, (y1 + y2) / 2

    w = (x2 - x1) * 1.2
    h = w * IMG_SIZE[1] / IMG_SIZE[0]

    margin_x, margin_y = w / 2, h / 2

    min_x, min_y = int(cx - margin_x), int(cy - margin_y)
    max_x, max_y = int(cx + margin_x), int(cy + margin_y)

    eye_rect = np.rint([min_x, min_y, max_x, max_y]).astype(int)

    eye_img = img[eye_rect[1]:eye_rect[3], eye_rect[0]:eye_rect[2]]

    return eye_img, eye_rect

# main

capture = cv2.VideoCapture(0)  # 디바이스 아이디
capture.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
capture.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

count = 1  # 이미지 파일 저장을 위한 카운터 초기화

while cv2.waitKey(10) < 0:  # wait를 늘리면 프레임이 줄어듬
  ret, img_ori = capture.read()
  img = img_ori.copy()
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  faces = detector(gray)
  face = faces[0]
  shapes = predictor(gray, face)
  shapes = face_utils.shape_to_np(shapes)

  eye_img_l, eye_rect_l = crop_eye(gray, eye_points=shapes[36:42])
  eye_img_r, eye_rect_r = crop_eye(gray, eye_points=shapes[42:48])

  eye_img_l = cv2.resize(eye_img_l, dsize=IMG_SIZE)
  eye_img_r = cv2.resize(eye_img_r, dsize=IMG_SIZE)
  eye_img_r = cv2.flip(eye_img_r, flipCode=1)

  # 이미지 저장 경로 및 파일명 설정
  
  path_l = f'closeEye1/{count}.png'
  cv2.imwrite(path_l, eye_img_l)
  count += 1  # 카운터 증가
  path_r = f'closeEye1/{count}.png'
  cv2.imwrite(path_r, eye_img_r)
  count += 1  # 카운터 증가
  

  eye_input_l = eye_img_l.copy().reshape((1, IMG_SIZE[1], IMG_SIZE[0], 1)).astype(np.float32) / 255.
  eye_input_r = eye_img_r.copy().reshape((1, IMG_SIZE[1], IMG_SIZE[0], 1)).astype(np.float32) / 255.

  pred_l = model.predict(eye_input_l)
  pred_r = model.predict(eye_input_r)

  # visualize
  state_l = 'O %.1f' if pred_l > 0.1 else '- %.1f'
  state_r = 'O %.1f' if pred_r > 0.1 else '- %.1f'

  state_l = state_l % pred_l
  state_r = state_r % pred_r

  cv2.rectangle(img, pt1=tuple(eye_rect_l[0:2]), pt2=tuple(eye_rect_l[2:4]), color=(255,255,255), thickness=2)
  cv2.rectangle(img, pt1=tuple(eye_rect_r[0:2]), pt2=tuple(eye_rect_r[2:4]), color=(255,255,255), thickness=2)

  cv2.putText(img, state_l, tuple(eye_rect_l[0:2]), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2)
  cv2.putText(img, state_r, tuple(eye_rect_r[0:2]), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2)
  cv2.imshow('result', img)
  capture.release()
  cv2.waitKey()
  cv2.destroyAllWindows()
  cv2.waitKey(1)

# capture.release()
# cv2.waitKey()
# cv2.destroyAllWindows()
# cv2.waitKey(1)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 26, 34, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 26, 34, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 13, 17, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 13, 17, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 6, 8, 64)          0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 6, 8, 128)         73856 

C:\Users\alsgu\AppData\Local\Temp\ipykernel_18860\2308023719.py:83: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  state_l = state_l % pred_l
C:\Users\alsgu\AppData\Local\Temp\ipykernel_18860\2308023719.py:84: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  state_r = state_r % pred_r


1/1 [==============================] - 0s 24ms/step


In [7]:
import cv2
import dlib
import numpy as np
from imutils import face_utils
from keras.models import load_model

IMG_SIZE = (34, 26)

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

model = load_model('models/2023_12_26_11_40_10.h5')
model.summary()

def crop_eye(img, eye_points):
    x1, y1 = np.amin(eye_points, axis=0)
    x2, y2 = np.amax(eye_points, axis=0)
    cx, cy = (x1 + x2) / 2, (y1 + y2) / 2

    w = (x2 - x1) * 1.2
    h = w * IMG_SIZE[1] / IMG_SIZE[0]

    margin_x, margin_y = w / 2, h / 2

    min_x, min_y = int(cx - margin_x), int(cy - margin_y)
    max_x, max_y = int(cx + margin_x), int(cy + margin_y)

    eye_rect = np.rint([min_x, min_y, max_x, max_y]).astype(int)

    eye_img = img[eye_rect[1]:eye_rect[3], eye_rect[0]:eye_rect[2]]

    return eye_img, eye_rect

# main

capture = cv2.VideoCapture(0)  # 디바이스 아이디
capture.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
capture.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

count = 341  # 이미지 파일 저장을 위한 카운터 초기화

while True:
    ret, img_ori = capture.read()
    
    if not ret:
        break

    img = img_ori.copy()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)

    if faces:
        face = faces[0]
        shapes = predictor(gray, face)
        shapes = face_utils.shape_to_np(shapes)

        eye_img_l, eye_rect_l = crop_eye(gray, eye_points=shapes[36:42])
        eye_img_r, eye_rect_r = crop_eye(gray, eye_points=shapes[42:48])

        eye_img_l = cv2.resize(eye_img_l, dsize=IMG_SIZE)
        eye_img_r = cv2.resize(eye_img_r, dsize=IMG_SIZE)
        eye_img_r = cv2.flip(eye_img_r, flipCode=1)

        # 이미지 저장 경로 및 파일명 설정
        path_l = f'openEye1/{count}.png'
        cv2.imwrite(path_l, eye_img_l)
        count += 1  # 카운터 증가
        path_r = f'openEye1/{count}.png'
        cv2.imwrite(path_r, eye_img_r)
        count += 1  # 카운터 증가

        eye_input_l = eye_img_l.copy().reshape((1, IMG_SIZE[1], IMG_SIZE[0], 1)).astype(np.float32) / 255.
        eye_input_r = eye_img_r.copy().reshape((1, IMG_SIZE[1], IMG_SIZE[0], 1)).astype(np.float32) / 255.

        pred_l = model.predict(eye_input_l)
        pred_r = model.predict(eye_input_r)

        # visualize
        state_l = 'O %.1f' if pred_l > 0.1 else '- %.1f'
        state_r = 'O %.1f' if pred_r > 0.1 else '- %.1f'

        state_l = state_l % pred_l
        state_r = state_r % pred_r

        cv2.rectangle(img, pt1=tuple(eye_rect_l[0:2]), pt2=tuple(eye_rect_l[2:4]), color=(255,255,255), thickness=2)
        cv2.rectangle(img, pt1=tuple(eye_rect_r[0:2]), pt2=tuple(eye_rect_r[2:4]), color=(255,255,255), thickness=2)

        cv2.putText(img, state_l, tuple(eye_rect_l[0:2]), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2)
        cv2.putText(img, state_r, tuple(eye_rect_r[0:2]), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2)
    
    cv2.imshow('result', img)
    
    key = cv2.waitKey(1)
    if key == 27:  # ESC 키를 누르면 종료
        break

capture.release()
cv2.destroyAllWindows()
cv2.waitKey(1)


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 26, 34, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 26, 34, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 13, 17, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 13, 17, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 6, 8, 64)          0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 6, 8, 128)         73856 

C:\Users\alsgu\AppData\Local\Temp\ipykernel_12740\3024266155.py:82: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  state_l = state_l % pred_l
C:\Users\alsgu\AppData\Local\Temp\ipykernel_12740\3024266155.py:83: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  state_r = state_r % pred_r


1/1 [==============================] - 0s 27ms/step


-1

## 데이터셋 만들기

In [9]:
!pip install scikit-learn


                                              0.0/9.2 MB ? eta -:--:--
     -----                                    1.3/9.2 MB 40.3 MB/s eta 0:00:01
     ----------                               2.5/9.2 MB 26.5 MB/s eta 0:00:01
     ---------------                          3.7/9.2 MB 29.2 MB/s eta 0:00:01
     --------------------                     4.8/9.2 MB 25.4 MB/s eta 0:00:01
     -------------------------                5.9/9.2 MB 25.0 MB/s eta 0:00:01
     ------------------------------           7.1/9.2 MB 25.1 MB/s eta 0:00:01
     -------------------------------          7.3/9.2 MB 26.1 MB/s eta 0:00:01
     ------------------------------------     8.3/9.2 MB 22.1 MB/s eta 0:00:01
     ---------------------------------------  9.2/9.2 MB 21.8 MB/s eta 0:00:01
     ---------------------------------------  9.2/9.2 MB 21.8 MB/s eta 0:00:01
     ---------------------------------------- 9.2/9.2 MB 19.7 MB/s eta 0:00:00
                                              0.0/44.1 MB ? 


[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
import os
from PIL import Image
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# 이미지를 읽고 지정된 크기로 조정하는 함수
def read_and_resize(file_path, target_size=(34, 26)):
    img = Image.open(file_path)
    img = img.resize(target_size, resample=Image.BICUBIC)
    return np.array(img)

# 각각의 이미지 파일이 저장된 폴더 경로
open_folder_path = "./openEye1/"
close_folder_path = "./closeEye1/"

# 각 폴더에 있는 이미지 파일들의 경로를 리스트로 수집
open_image_files = [os.path.join(open_folder_path, f) for f in os.listdir(open_folder_path) if f.endswith(".png")]
close_image_files = [os.path.join(close_folder_path, f) for f in os.listdir(close_folder_path) if f.endswith(".png")]

# 이미지 데이터를 읽어서 리스트에 추가
image_data = []
state = []

# 눈 뜬 사진 추가
for open_file in open_image_files:
    open_data = read_and_resize(open_file)
    image_data.append(open_data)
    state.append("open")

# 눈 감은 사진 추가
for close_file in close_image_files:
    close_data = read_and_resize(close_file)
    image_data.append(close_data)
    state.append("close")

# 데이터셋 생성
data = pd.DataFrame({"state": state, "image": image_data})

# 데이터를 랜덤하게 섞은 후 훈련 데이터와 검증 데이터로 분할
train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

# CSV 파일로 저장
print("image_data", np.array(image_data))
print("data", data)
data.to_csv("newDataset/dataset.csv", index=False)
train_data.to_csv("newDataset/train_dataset.csv")
val_data.to_csv("newDataset/val_dataset.csv")

# 이미지 데이터를 NumPy 배열로 변환
x_train = np.array(train_data["image"].tolist()).astype(np.float32)
x_val = np.array(val_data["image"].tolist()).astype(np.float32)

# 레이블을 NumPy 배열로 변환
y_train = np.array(train_data["state"].map({"open": 1, "close": 0})).reshape(-1, 1).astype(np.float32)
y_val = np.array(val_data["state"].map({"open": 1, "close": 0})).reshape(-1, 1).astype(np.float32)

# NumPy 배열을 파일로 저장
np.save("newDataset/x_train.npy", x_train)
np.save("newDataset/x_val.npy", x_val)
np.save("newDataset/y_train.npy", y_train)
np.save("newDataset/y_val.npy", y_val)

# 출력 확인
print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)


image_data [[[108  96  90 ...  35  37  37]
  [ 99  87  89 ...  28  28  32]
  [100  94  97 ...  70  63  53]
  ...
  [119 119 119 ... 107 105 112]
  [122 120 119 ... 113 109 113]
  [119 118 117 ... 116 114 114]]

 [[ 88  87  89 ...  34  31  36]
  [ 94  92  91 ...  46  31  31]
  [ 93  93  93 ...  90  74  57]
  ...
  [104 104 102 ... 107 105 103]
  [106 105 105 ... 108 105 106]
  [106 105 106 ... 107 106 108]]

 [[ 98  88  89 ...  44  44  48]
  [ 89  85  90 ...  78  62  45]
  [ 97  97  95 ... 106 111 109]
  ...
  [123 121 119 ... 122 119 118]
  [122 121 121 ... 122 120 120]
  [124 121 122 ... 123 120 120]]

 ...

 [[171 175 176 ... 138 131 130]
  [176 175 175 ... 137 135 129]
  [174 171 175 ... 130 131 129]
  ...
  [199 199 198 ... 160 166 162]
  [199 197 195 ... 162 161 162]
  [201 198 199 ... 164 163 166]]

 [[179 177 182 ... 128 128 128]
  [183 180 186 ... 126 126 127]
  [182 185 185 ... 127 129 130]
  ...
  [215 212 211 ... 194 193 192]
  [218 215 214 ... 190 186 194]
  [221 215 214 ..

## 제스쳐 인식

In [38]:
import cv2
from utils_display import DisplayHand
from utils_mediapipe import MediaPipeHand
from utils_joint_angle import GestureRecognition
import argparse

# ArgumentParser 생성
parser = argparse.ArgumentParser()
parser.add_argument('-m', '--mode', default='eval', help='train / eval')
args, unknown_args = parser.parse_known_args()

# mode 변수 설정
mode = args.mode

# mediapipe hand 클래스 로드
pipe = MediaPipeHand(static_image_mode=False, max_num_hands=2)

# 디스플레이 클래스 로드
disp = DisplayHand(max_num_hands=2)

# 비디오 캡처 시작
cap = cv2.VideoCapture(0)
# cap.set(cv2.CAP_PROP_FRAME_WIDTH, 2560)
# cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 1440)

# 제스처 인식 클래스 로드
gest = GestureRecognition(mode)

counter = 0
class_label = 0
while cap.isOpened():
    ret, img = cap.read()
    if not ret:
        break

    # 3인칭 뷰를 위해 이미지 뒤집기
    img = cv2.flip(img, 1)

    # 참조로 전달하기 위해 이미지를 쓸 수 없게 표시(옵션)
    img.flags.writeable = False

    # 키포인트를 추출하기 위해 피드포워드
    param = pipe.forward(img)
    if (param[0]['class'] is not None) and (mode=='eval'):
        param[0]['gesture'] = gest.eval(param[0]['angle'])

    img.flags.writeable = True

    # 키포인트 표시
    cv2.imshow('img 2D', disp.draw2d(img.copy(), param))

    key = cv2.waitKey(1)
    if key==27:
        break
    if key==97 and (param[0]['class'] is not None) and (mode=='train'):
        print('train mode')
        class_label = (class_label + 1) % 11
        print('제스처로 변경:', list(gest.gesture)[class_label])
    if key==32 and (param[0]['class'] is not None) and (mode=='train'):
        # 스페이스바를 눌러 훈련 데이터 기록
        gest.train(param[0]['angle'], class_label)
        print('저장됨:', list(gest.gesture)[class_label], '카운터:', counter) # 각 클래스에 대해 약 10번 기록
        counter += 1
    if key==32 and (param[0]['class'] is not None) and (mode=='eval'):
        cv2.waitKey(0) # 사용자가 아무 키나 누를 때까지 디스플레이 일시 중지        

pipe.pipe.close()
cap.release()
cv2.destroyAllWindows()


NameError: name 'audio_classifier' is not defined

In [37]:
!pip install opencv-python numpy


In [35]:
!pip install --upgrade mediapipe


## 제스처 음성 출력

In [1]:
import cv2
from utils_display import DisplayHand
from utils_mediapipe import MediaPipeHand
from utils_joint_angle import GestureRecognition
import numpy as np
import time
import pyttsx3
import argparse

# ArgumentParser 생성
parser = argparse.ArgumentParser()
parser.add_argument('-m', '--mode', default='eval', help='train / eval')
args, unknown_args = parser.parse_known_args()

# mode 변수 설정
mode = args.mode

# mediapipe hand 클래스 로드
pipe = MediaPipeHand(static_image_mode=False, max_num_hands=1)

# 디스플레이 클래스 로드
disp = DisplayHand(max_num_hands=1)

# 비디오 캡처 시작
cap = cv2.VideoCapture(0)

# 제스처 인식 클래스 로드
gest = GestureRecognition(mode)

# 음성 출력 엔진 초기화
engine = pyttsx3.init()

# 'home' 제스처가 인식된 시간을 저장하는 변수
home_start_time = None

while cap.isOpened():
    ret, img = cap.read()
    if not ret:
        break

    # 3인칭 뷰를 위해 이미지 뒤집기
    img = cv2.flip(img, 1)

    # 참조로 전달하기 위해 이미지를 쓸 수 없게 표시(옵션)
    img.flags.writeable = False

    # 키포인트를 추출하기 위해 피드포워드
    param = pipe.forward(img)
    
    if (param[0]['class'] is not None) and (mode == 'eval'):
        param[0]['gesture'] = gest.eval(param[0]['angle'])
        
        # 'home' 제스처가 인식되면 음성 출력 및 시간 기록
        if param[0]['gesture'] == 'home':
            if home_start_time is None:
                home_start_time = time.time()
            else:
                elapsed_time = time.time() - home_start_time
                if elapsed_time >= 3:  # 3초 이상 동안 'home' 제스처가 인식되면
                    engine.say('안내를 시작합니다')
                    engine.runAndWait()
                    home_start_time = None  # 초기화
        else:
            home_start_time = None  # 'home' 제스처가 아닌 경우 초기화

    img.flags.writeable = True

    # 키포인트 표시
    cv2.imshow('img 2D', disp.draw2d(img.copy(), param))

    key = cv2.waitKey(1)
    if key == 27:
        break

engine.stop()
pipe.pipe.close()
cap.release()
cv2.destroyAllWindows()


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.



# 최종

In [26]:
import cv2
from utils_display import DisplayHand
from utils_mediapipe import MediaPipeHand
from utils_joint_angle import GestureRecognition
import dlib
import numpy as np
from imutils import face_utils
from tensorflow.keras.models import load_model
import pyttsx3
import time
import argparse

IMG_SIZE = (34, 26)

# ArgumentParser 생성
parser = argparse.ArgumentParser()
parser.add_argument('-m', '--mode', default='eval', help='train / eval')
args, unknown_args = parser.parse_known_args()

# mode 변수 설정
mode = args.mode

# mediapipe hand 클래스 로드
pipe = MediaPipeHand(static_image_mode=False, max_num_hands=1)

# 디스플레이 클래스 로드
disp = DisplayHand(max_num_hands=1)

# 비디오 캡처 시작
cap = cv2.VideoCapture(0)
# cap.set(cv2.CAP_PROP_FRAME_WIDTH, 2560)
# cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 1440)

# 제스처 인식 클래스 로드
gest = GestureRecognition(mode)

# 음성 출력 엔진 초기화
engine = pyttsx3.init()
engine.setProperty('rate', 150)  # 음성 속도 조절

# 'home' 제스처가 인식된 시간을 저장하는 변수
home_start_time = None

# dlib face detector 및 shape predictor 초기화
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

# 눈 깜빡임 주기 확인을 위한 변수들
blink_threshold = 3  # 눈을 깜빡이지 않아도 작동할 시간
blink_counter = 0    # 눈을 깜빡인 시간을 측정하는 카운터

# 딥러닝 모델 로드
model = load_model('models/2024_01_05_14_16_21.h5')
model.summary()

# 이전에 눈을 감은 시간
prev_time = time.time()

def crop_eye(img, eye_points):
    x1, y1 = np.amin(eye_points, axis=0)
    x2, y2 = np.amax(eye_points, axis=0)
    cx, cy = (x1 + x2) / 2, (y1 + y2) / 2

    w = (x2 - x1) * 1.2
    h = w * IMG_SIZE[1] / IMG_SIZE[0]

    margin_x, margin_y = w / 2, h / 2

    min_x, min_y = int(cx - margin_x), int(cy - margin_y)
    max_x, max_y = int(cx + margin_x), int(cy + margin_y)

    eye_rect = np.rint([min_x, min_y, max_x, max_y]).astype(int)

    eye_img = img[eye_rect[1]:eye_rect[3], eye_rect[0]:eye_rect[2]]

    return eye_img, eye_rect

while cap.isOpened():
    ret, img_ori = cap.read()
    if not ret:
        break

    img = img_ori.copy()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # 3인칭 뷰를 위해 이미지 뒤집기
    #img = cv2.flip(img, 1)

    # 참조로 전달하기 위해 이미지를 쓸 수 없게 표시(옵션)
    img.flags.writeable = False

    # 키포인트를 추출하기 위해 피드포워드
    param = pipe.forward(img)

    if (param[0]['class'] is not None) and (mode == 'eval'):
        param[0]['gesture'] = gest.eval(param[0]['angle'])

        # 'home' 제스처가 인식되면 음성 출력 및 시간 기록
        if param[0]['gesture'] == 'home':
            if home_start_time is None:
                home_start_time = time.time()
            else:
                elapsed_time = time.time() - home_start_time
                if elapsed_time >= 3:  # 3초 이상 동안 'home' 제스처가 인식되면
                    engine.say('안내를 시작합니다')
                    engine.runAndWait()
                    home_start_time = None  # 초기화
        else:
            home_start_time = None  # 'home' 제스처가 아닌 경우 초기화

    img.flags.writeable = True

    # 키포인트 표시
    img_2d = disp.draw2d(img.copy(), param)

    # dlib 얼굴 및 랜드마크 검출
    faces = detector(gray)

    if len(faces) > 0:
        face = faces[0]
        shapes = predictor(gray, face)
        shapes = face_utils.shape_to_np(shapes)

        eye_img_l, eye_rect_l = crop_eye(gray, eye_points=shapes[36:42])
        eye_img_r, eye_rect_r = crop_eye(gray, eye_points=shapes[42:48])

        eye_img_l = cv2.resize(eye_img_l, dsize=IMG_SIZE)
        eye_img_r = cv2.resize(eye_img_r, dsize=IMG_SIZE)

        # 눈 이미지 수평으로 뒤집기
        eye_img_l_flipped = cv2.flip(eye_img_l, 1)
        eye_img_r_flipped = cv2.flip(eye_img_r, 1)

        eye_input_l = eye_img_l.copy().reshape((1, IMG_SIZE[1], IMG_SIZE[0], 1)).astype(np.float32) / 255.
        eye_input_r = eye_img_r.copy().reshape((1, IMG_SIZE[1], IMG_SIZE[0], 1)).astype(np.float32) / 255.

        pred_l = model.predict(eye_input_l)
        pred_r = model.predict(eye_input_r)

        # 눈 깜빡임 주기 확인
        if pred_l < 0.2 and pred_r < 0.2:
            blink_counter += 1
            if blink_counter >= blink_threshold:
                engine.say("졸음 운전 감지")
                engine.runAndWait()
                blink_counter = 0  # 초기화
        else:
            blink_counter = 0  # 초기화

        # visualize
        state_l = 'O %.1f' if pred_l > 0.1 else '- %.1f'
        state_r = 'O %.1f' if pred_r > 0.1 else '- %.1f'

        state_l = state_l % pred_l
        state_r = state_r % pred_r

        cv2.rectangle(img, pt1=tuple(eye_rect_l[0:2]), pt2=tuple(eye_rect_l[2:4]), color=(255, 255, 255), thickness=2)
        cv2.rectangle(img, pt1=tuple(eye_rect_r[0:2]), pt2=tuple(eye_rect_r[2:4]), color=(255, 255, 255), thickness=2)

        cv2.putText(img, state_l, tuple(eye_rect_l[0:2]), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        cv2.putText(img, state_r, tuple(eye_rect_r[0:2]), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

    # 이미지 합치기
    combined_img = cv2.addWeighted(img_2d, 0.5, img, 0.5, 0)

    cv2.imshow('result', combined_img)  # 두 결과를 합쳐서 표시하는 윈도우

    key = cv2.waitKey(1)
    if key == 27:
        break

engine.stop()
pipe.pipe.close()
cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)


NameError: name 'audio_classifier' is not defined

# 지도 호출

In [10]:
!pip install selenium

  Obtaining dependency information for selenium from https://files.pythonhosted.org/packages/dc/72/96b5afa16908f9abc7c24b70adfd3a46c9740eb728ddfeab28379e38eaf9/selenium-4.16.0-py3-none-any.whl.metadata
  Obtaining dependency information for trio~=0.17 from https://files.pythonhosted.org/packages/14/fb/9299cf74953f473a15accfdbe2c15218e766bae8c796f2567c83bae03e98/trio-0.24.0-py3-none-any.whl.metadata
  Obtaining dependency information for trio-websocket~=0.9 from https://files.pythonhosted.org/packages/48/be/a9ae5f50cad5b6f85bd2574c2c923730098530096e170c1ce7452394d7aa/trio_websocket-0.11.1-py3-none-any.whl.metadata
  Obtaining dependency information for outcome from https://files.pythonhosted.org/packages/55/8b/5ab7257531a5d830fc8000c476e63c935488d74609b50f9384a643ec0a62/outcome-1.3.0.post0-py2.py3-none-any.whl.metadata
  Using cached sniffio-1.3.0-py3-none-any.whl (10 kB)
  Using cached wsproto-1.2.0-py3-none-any.whl (24 kB)
  Using cached h11-0.14.0-py3-none-any.whl (58 kB)
   --------

In [2]:
import requests
import folium
import json
import os
import webbrowser
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time 
from datetime import datetime, timedelta

# 중간 지점 계산 함수
def calculate_midpoint(lat1, lon1, lat2, lon2):
    return ((lat1 + lat2) / 2, (lon1 + lon2) / 2)

# https://ipstack.com
##########ip주소로 위도, 경도 찾는 API##########
key = 'c887f69dd63ddb1cd9d4f53408974e46'
send_url = 'http://api.ipstack.com/check?access_key=' + key

# tmap app key
APP_KEY = "HpVe5crDV38GvHu7tUyMA5CSmhca5PQk3vRWkFH0"

headers = {
    "appKey": APP_KEY
}
# ip 주소 위치에 따른 위도 경도
r = requests.get(send_url)
j = json.loads(r.text)

lat = j['latitude'] # 위도
lon = j['longitude'] # 경도

start = str(lat) + ", " + str(lon)

# 도착지의 위도와 경도
end_lat =  37.28406376153335
end_lon = 126.85166240118258

end = str(end_lat) + ", " + str(end_lon)

# 중간 지점 계산
midpoint_lat, midpoint_lon = calculate_midpoint(lat, lon, end_lat, end_lon)


# 티맵 경로 안내 API URL
url = f"https://api2.sktelecom.com/tmap/routes?version=1&format=json&startX={start.split(',')[1]}&startY={start.split(',')[0]}&endX={end.split(',')[1]}&endY={end.split(',')[0]}"


response = requests.get(url, headers=headers)       

# 응답이 성공적인지 확인합니다.
if response.status_code == 200:
    # API 응답에서 경로 정보를 추출합니다.
    routes_info = response.json()["features"]

    # 경로를 그릴 좌표들의 리스트를 생성합니다.
    path_coordinates = []
    for feature in routes_info:
        if feature["geometry"]["type"] == "LineString":
            # 좌표계가 [경도, 위도] 형태로 되어 있으므로, [위도, 경도] 순으로 바꿔서 저장합니다.
            path_coordinates.extend([[coord[1], coord[0]] for coord in feature["geometry"]["coordinates"]])

    # 예상 이동 시간을 추출합니다.
    total_time = sum([feature["properties"]["totalTime"] for feature in routes_info if "totalTime" in feature["properties"]])

    # 현재 시각을 구합니다.
    now = datetime.now()
    
    # 현재 시각에 total_time초를  더합니다.
    future_time = now + timedelta(seconds=total_time)
    
    # 도착 시간 저장
    arrivalTime =  f"예상 도착 시간:{future_time.hour}시 {future_time.minute}분"
    
    # 초 단위로 얻은 소요 시간을 시, 분, 초로 나누어 저장
    if total_time > 60:
        min = total_time // 60
        sec = total_time % 60
        if min > 60:
            hour = min // 60
            min = min % 60
            total_time = str(hour) + "시간 " + str(min) + "분 " + str(sec) + "초"
        else:
            total_time = str(min) + "분 " + str(sec) + "초"
    else:
        total_time = str(total_time) + "초"
    requireTime = f"예상 이동 시간: {total_time}"
    
    #마커 생성
    start_marker = folium.Marker(location=path_coordinates[0], popup=folium.Popup('<strong>현재위치</strong><br>'+requireTime, max_width=300, show=True), icon=folium.Icon(color='green'))
    end_marker = folium.Marker(location=path_coordinates[-1], popup=folium.Popup('<strong>집</strong><br>'+arrivalTime, max_width=300, show=True), icon=folium.Icon(color='red'))

    #km 계산
    total_distance = sum([feature["properties"]["totalDistance"] for feature in routes_info if "totalDistance" in feature["properties"]])
    total_distance_km = total_distance / 1000  # 미터를 킬로미터로 변환

    if 0 <= total_distance_km <= 15:
        zoom_level = 13
    elif 15.1 <= total_distance_km <= 30:
        zoom_level = 12
    elif 30.1 <= total_distance_km <= 110:
        zoom_level = 11
    elif 110.1 <= total_distance_km <= 230:
        zoom_level = 10
    elif 230.1 <= total_distance_km <= 400:
        zoom_level = 9
    else:
        zoom_level = 8
    
    # Folium 지도 객체를 생성합니다.
    # 출발지의 위도와 경도를 중심으로 지도를 생성합니다.
    map = folium.Map(location=[midpoint_lat, midpoint_lon], zoom_start=zoom_level)

    # 출발지와 도착지 마커를 지도에 추가합니다.
    start_marker.add_to(map)
    end_marker.add_to(map)
    
    # Folium의 PolyLine을 사용하여 지도에 선을 그립니다.
    folium.PolyLine(path_coordinates, color="red", weight=3, opacity=1).add_to(map)

    # 지도를 HTML 파일로 저장합니다.
    map.save("route.html")
    
    # 총 거리 추출 및 km 단위로 변환
    print(f"총 거리: {total_distance_km:.2f} km")
    
    #webbrowser.open("route.html")
    
    # 크롬 드라이버를 설정합니다.
    driver = webdriver.Chrome()
    
    #브라우저 창 크기 설정
    driver.set_window_size(1920, 1080)
    
    # HTML 파일의 경로를 지정합니다. 파일이 위치한 경로를 정확하게 입력해야 합니다.
    file_path = 'file://' + os.path.realpath('route.html')

    # 크롬에서 HTML 파일을 엽니다.
    driver.get(file_path)

    # 5초간 대기합니다.
    time.sleep(10)

    # 현재 탭을 닫습니다.
    driver.close()
    
else:
    print("Error:", response.status_code)


총 거리: 46.32 km


# 찐최종

In [5]:
import cv2
from utils_display import DisplayHand
from utils_mediapipe import MediaPipeHand
from utils_joint_angle import GestureRecognition
import dlib
import numpy as np
from imutils import face_utils
from tensorflow.keras.models import load_model
import pyttsx3
import time
import argparse
from tmap import route

def crop_eye(img, eye_points):
    x1, y1 = np.amin(eye_points, axis=0)
    x2, y2 = np.amax(eye_points, axis=0)
    cx, cy = (x1 + x2) / 2, (y1 + y2) / 2

    w = (x2 - x1) * 1.2
    h = w * IMG_SIZE[1] / IMG_SIZE[0]

    margin_x, margin_y = w / 2, h / 2

    min_x, min_y = int(cx - margin_x), int(cy - margin_y)
    max_x, max_y = int(cx + margin_x), int(cy + margin_y)

    eye_rect = np.rint([min_x, min_y, max_x, max_y]).astype(int)

    eye_img = img[eye_rect[1]:eye_rect[3], eye_rect[0]:eye_rect[2]]

    return eye_img, eye_rect


# 출발지 도착지 위,경도 중간 지점 계산 함수
def calculate_midpoint(lat1, lon1, lat2, lon2):
    return ((lat1 + lat2) / 2, (lon1 + lon2) / 2)

IMG_SIZE = (34, 26)

# ArgumentParser 생성
parser = argparse.ArgumentParser()
parser.add_argument('-m', '--mode', default='eval', help='train / eval')
args, unknown_args = parser.parse_known_args()

# mode 변수 설정
mode = args.mode

# mediapipe hand 클래스 로드
pipe = MediaPipeHand(static_image_mode=False, max_num_hands=1)

# 딥러닝 모델 로드
model = load_model('models/2024_01_05_14_16_21.h5')
model.summary()

# 디스플레이 클래스 로드
disp = DisplayHand(max_num_hands=1)

# dlib face detector 및 shape predictor 초기화
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

# 비디오 캡처 시작
cap = cv2.VideoCapture(0)
# cap.set(cv2.CAP_PROP_FRAME_WIDTH, 2560)
# cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 1440)

# 제스처 인식 클래스 로드
gest = GestureRecognition(mode)

# 음성 출력 엔진 초기화
engine = pyttsx3.init()
engine.setProperty('rate', 150)  # 음성 속도 조절

# 'home' 제스처가 인식된 시간을 저장하는 변수
home_start_time = None

# 눈 깜빡임 주기 확인을 위한 변수들
blink_threshold = 3  # 눈을 깜빡이지 않아도 작동할 시간
blink_counter = 0    # 눈을 깜빡인 시간을 측정하는 카운터

# 이전에 눈을 감은 시간
prev_time = time.time()

while cap.isOpened():
    ret, img_ori = cap.read()
    if not ret:
        break

    img = img_ori.copy()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # 참조로 전달하기 위해 이미지를 쓸 수 없게 표시
    img.flags.writeable = False

    # 키포인트를 추출하기 위해 피드포워드
    param = pipe.forward(img)

    if (param[0]['class'] is not None) and (mode == 'eval'):
        param[0]['gesture'] = gest.eval(param[0]['angle'])

        # 'home' 제스처가 인식되면 음성 출력 및 시간 기록
        if param[0]['gesture'] == 'home':
            if home_start_time is None:
                home_start_time = time.time()
            else:
                elapsed_time = time.time() - home_start_time
                if elapsed_time >= 3:  # 3초 이상 동안 'home' 제스처가 인식되면
                    engine.say('안내를 시작합니다')
                    engine.runAndWait()
                    home_start_time = None  # 초기화
                    route()
                    
        else:
            home_start_time = None  # 'home' 제스처가 아닌 경우 초기화

    img.flags.writeable = True

    # 키포인트 표시
    img_2d = disp.draw2d(img.copy(), param)

    # dlib 얼굴 및 랜드마크 검출
    faces = detector(gray)

    if len(faces) > 0:
        face = faces[0]
        shapes = predictor(gray, face)
        shapes = face_utils.shape_to_np(shapes)

        eye_img_l, eye_rect_l = crop_eye(gray, eye_points=shapes[36:42])
        eye_img_r, eye_rect_r = crop_eye(gray, eye_points=shapes[42:48])

        eye_img_l = cv2.resize(eye_img_l, dsize=IMG_SIZE)
        eye_img_r = cv2.resize(eye_img_r, dsize=IMG_SIZE)

        # 눈 이미지 수평으로 뒤집기
        eye_img_l_flipped = cv2.flip(eye_img_l, 1)
        eye_img_r_flipped = cv2.flip(eye_img_r, 1)

        eye_input_l = eye_img_l.copy().reshape((1, IMG_SIZE[1], IMG_SIZE[0], 1)).astype(np.float32) / 255.
        eye_input_r = eye_img_r.copy().reshape((1, IMG_SIZE[1], IMG_SIZE[0], 1)).astype(np.float32) / 255.

        pred_l = model.predict(eye_input_l)
        pred_r = model.predict(eye_input_r)

        # 눈 깜빡임 주기 확인
        if pred_l < 0.2 and pred_r < 0.2:
            blink_counter += 1
            if blink_counter >= blink_threshold:
                engine.say("졸음 운전 감지")
                engine.runAndWait()
                blink_counter = 0  # 초기화
        else:
            blink_counter = 0  # 초기화

        # visualize
        state_l = 'O %.1f' if pred_l > 0.1 else '- %.1f'
        state_r = 'O %.1f' if pred_r > 0.1 else '- %.1f'

        state_l = state_l % pred_l
        state_r = state_r % pred_r

        cv2.rectangle(img, pt1=tuple(eye_rect_l[0:2]), pt2=tuple(eye_rect_l[2:4]), color=(255, 255, 255), thickness=2)
        cv2.rectangle(img, pt1=tuple(eye_rect_r[0:2]), pt2=tuple(eye_rect_r[2:4]), color=(255, 255, 255), thickness=2)

        cv2.putText(img, state_l, tuple(eye_rect_l[0:2]), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        cv2.putText(img, state_r, tuple(eye_rect_r[0:2]), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

    # 이미지 합치기
    combined_img = cv2.addWeighted(img_2d, 0.5, img, 0.5, 0)

    cv2.imshow('result', combined_img)

    key = cv2.waitKey(1)
    if key == 27:
        break

engine.stop()
pipe.pipe.close()
cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 26, 34, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 26, 34, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 13, 17, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 13, 17, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 6, 8, 64)          0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 6, 8, 128)         73856 

-1